In [2]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import pNN_FA as pNN

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

args = parser.parse_args([])
args = FormulateArgs(args)

valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
pprint.pprint(datainfo)

for x,y in valid_loader:
    X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
for x,y in test_loader:
    X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [3]:
topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
pnn = pNN.pNN(topology, args).to(args.DEVICE)
pnn.UpdateVariation(1, 0.)

modelname = f"data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{args.e_train}_dropout_{args.dropout}.model"
trained_model = torch.load(f'./models/{modelname}')
trained_model.UpdateVariation(1, 0.)
trained_model.UpdateDropout(0.)

for i, j in zip(trained_model.model, pnn.model):
    j.theta_.data = i.theta_.data

In [4]:
pnn.UpdateFault(1, 0)

In [7]:
pred = pnn(X_valid)[0,0,:,:]
(torch.argmax(pred, dim=1) == y_valid).sum() / y_valid.numel()

tensor(1.)

In [15]:
pnn.UpdateFault(1, 2)

In [17]:
pred = pnn(X_valid)[0,0,:,:]
(torch.argmax(pred, dim=1) == y_valid).sum() / y_valid.numel()

tensor(0.7826)

In [16]:




if not os.path.exists(f"./evaluation/result_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{args.e_train}_dropout_{args.dropout}.matrix"):

    N_Faults = 500
    results = torch.zeros([N_Faults, 6])
    evaluator = Evaluator(args).to(args.DEVICE)
    modelname = f"data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{args.e_train}_faults_{args.e_fault:1d}.model"

    if os.path.exists(f'./models/{modelname}'):
        model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
        print(modelname)
        model.UpdateArgs(args)
        
        model.UpdateVariation(args.N_test, args.e_test)
        model.UpdateFault(1, args.e_fault_test)
    
        SetSeed(args.SEED)

        for faultsample in range(N_Faults):
            print(faultsample)
        
            with torch.no_grad():
                result_valid = evaluator(model, X_valid, y_valid)
                valid_acc, _, valid_power, valid_area = result_valid['acc'], result_valid['std'], result_valid['power'], result_valid['area']
                result_test  = evaluator(model, X_test,  y_test)
                test_acc, _, test_power, test_area = result_test['acc'], result_test['std'], result_test['power'], result_test['area']
        
            results[faultsample, 0] = valid_acc
            results[faultsample, 1] = valid_power
            results[faultsample, 2] = valid_area
            results[faultsample, 3] = test_acc
            results[faultsample, 4] = test_power
            results[faultsample, 5] = test_area
                
                            
        torch.save(results, f"./evaluation/result_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}_eTrain_{args.e_train}_eTest{args.e_test}_fTrain_{args.e_fault}_fTest{args.e_fault_test}.matrix")

data_00_acuteinflammation_seed_00_epsilon_0.1_faults_1.model


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [14]:
results[:,3].mean()

tensor(0.9741)

In [19]:
results[:,3].mean()

tensor(0.8563)